<a href="https://colab.research.google.com/github/doironm1ATWIT/Applied-Deep-Learning-Final-Project/blob/main/Applied_Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel




from google.colab import files
uploaded = files.upload()

Saving humor_detection_data.csv to humor_detection_data.csv


In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['humor_detection_data.csv']))

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
df = df.sample(1000)

In [6]:
def preprocess_data(df):
  inputs = []
  for text in df['text']:
    encoded_input = tokenizer.encode(text, add_special_tokens=True, max_length = 100, truncation = True)
    inputs.append(encoded_input)

  padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, padding = 'post', maxlen=100)

  return padded_inputs

In [7]:
inputs = preprocess_data(df)

In [8]:
number_of_sent = len(df)

In [9]:
def creating_model(df):

  #input layers
  input_sent = tf.keras.Input(shape=(100,), dtype='int32')

  #initialize embeddings
  embeddings = bert_model(input_sent)[0]
  #Initialize paralell lines list
  parallel_lines = []

  #for i in range of dataframe create sentence embedding add dense layer (paralell line)
  for i in range(df):

    sentence_embedding = embeddings[:, :, :]


    parallel_line = tf.keras.layers.Dense(20, activation = 'relu')(sentence_embedding)
    parallel_lines.append(parallel_line)

  #concat the parallel lines
  concat_layer = tf.keras.layers.Concatenate(axis=1)(parallel_lines)

  #sequential model
  sequential_layers = tf.keras.Sequential([
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])

  output = sequential_layers(concat_layer)

  model = tf.keras.Model(inputs=input_sent, outputs = output)

  return model

In [10]:
X_train, X_test,y_train, y_test = train_test_split(inputs, df['humor'], test_size = 0.3, random_state=0)

In [11]:
model = creating_model(number_of_sent)

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=6, validation_split=0.1,batch_size = 1)

Epoch 1/6


630/630 [==============================] - 744s 339ms/step - loss: 0.6976 - accuracy: 0.5136 - val_loss: 0.6942 - val_accuracy: 0.4858
Epoch 2/6
630/630 [==============================] - 191s 303ms/step - loss: 0.6917 - accuracy: 0.5390 - val_loss: 0.6976 - val_accuracy: 0.4857
Epoch 3/6
630/630 [==============================] - 190s 301ms/step - loss: 0.6912 - accuracy: 0.5410 - val_loss: 0.6961 - val_accuracy: 0.4861
Epoch 4/6
630/630 [==============================] - 189s 300ms/step - loss: 0.6913 - accuracy: 0.5402 - val_loss: 0.6979 - val_accuracy: 0.4857
Epoch 5/6
630/630 [==============================] - 188s 298ms/step - loss: 0.6906 - accuracy: 0.5412 - val_loss: 0.6982 - val_accuracy: 0.4857
Epoch 6/6
630/630 [==============================] - 189s 300ms/step - loss: 0.6916 - accuracy: 0.5406 - val_loss: 0.6995 - val_accuracy: 0.4857


In [13]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save the model
model.save("bert_humor_detection_model")

10/10 [==============================] - 37s 343ms/step - loss: 0.7011 - accuracy: 0.4767
Test Loss: 0.7011, Test Accuracy: 0.4767
